In [3]:
import mlflow
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
from mlflow.models import infer_signature

In [5]:
# Set the tracking URI to mlflow server

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
# Load the dataset

X,y = datasets.load_iris(return_X_y=True)


In [9]:
# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
# Define model hyperparameters
params = {"solver":"newton-cg", "multi_class":"auto", "max_iter":1000, "random_state":1000}

In [23]:
# Train the model

lr = LogisticRegression(**params)

lr.fit(X_train, y_train)

/Users/mohammedanas/Learning/Machine Learning/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [24]:
# Make predictions

y_pred = lr.predict(X_test)
y_pred

array([2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2,
       1, 0, 0, 2, 0, 2, 1, 2])

In [25]:
accuracy = accuracy_score(y_test, y_pred)

In [26]:
accuracy

1.0

In [27]:
# MLFlow Tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Start a new MLflow experiment
mlflow.set_experiment("Iris_Classification")

# Start an MLflow run
with mlflow.start_run():
    # Log model hyperparameters
    mlflow.log_params(params)

    # Log model accuracy metrics
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training info", "Basic LR model for iris dataset")
    
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="Iris_Classification_Model",
    )

2025/05/20 20:19:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'Iris_Classification_Model' already exists. Creating a new version of this model...
2025/05/20 20:19:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Iris_Classification_Model, version 2


🏃 View run blushing-kite-138 at: http://127.0.0.1:5000/#/experiments/709085634117900399/runs/88e6163b03544a34954e46e9ea2a7224
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/709085634117900399


Created version '2' of model 'Iris_Classification_Model'.


## Inferencing and Validating Model

In [46]:
import mlflow
from mlflow.models import Model

model_uri = 'runs:/88e6163b03544a34954e46e9ea2a7224/iris_model'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

input_data

array([[6. , 2.9, 4.5, 1.5],
       [4.9, 3. , 1.4, 0.2],
       [6.8, 3. , 5.5, 2.1],
       [7.7, 2.8, 6.7, 2. ],
       [6. , 2.7, 5.1, 1.6],
       [6.7, 3.3, 5.7, 2.1],
       [4.9, 2.5, 4.5, 1.7],
       [7.4, 2.8, 6.1, 1.9],
       [5.7, 2.5, 5. , 2. ],
       [5.6, 2.5, 3.9, 1.1],
       [6.8, 2.8, 4.8, 1.4],
       [5.4, 3.4, 1.7, 0.2],
       [6.4, 3.2, 4.5, 1.5],
       [5.2, 4.1, 1.5, 0.1],
       [6.5, 2.8, 4.6, 1.5],
       [7.7, 3.8, 6.7, 2.2],
       [5.4, 3.4, 1.5, 0.4],
       [4.8, 3.4, 1.6, 0.2],
       [6.4, 3.1, 5.5, 1.8],
       [6.7, 3.1, 4.4, 1.4],
       [6.3, 2.5, 5. , 1.9],
       [4.4, 3. , 1.3, 0.2],
       [6.3, 3.4, 5.6, 2.4],
       [4.3, 3. , 1.1, 0.1],
       [5. , 3.4, 1.5, 0.2],
       [5.2, 3.5, 1.5, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [6.9, 3.2, 5.7, 2.3],
       [6.3, 2.9, 5.6, 1.8],
       [5.5, 2.5, 4. , 1.3],
       [6.7, 3.1, 4.7, 1.5],
       [6.9, 3.1, 4.9, 1.5],
       [5. , 3.4, 1.6, 0.4],
       [4.4, 3.2, 1.3, 0.2],
       [5.5, 2

In [51]:

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

2025/05/21 14:55:29 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/05/21 14:55:29 INFO mlflow.utils.virtualenv: Environment /var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmp183tk0k9/envs/virtualenv_envs/mlflow-3371859241c0371f405bc46b1b817b0ba908bc2c already exists
2025/05/21 14:55:29 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmp183tk0k9/envs/virtualenv_envs/mlflow-3371859241c0371f405bc46b1b817b0ba908bc2c/bin/activate && python -c ""']'
2025/05/21 14:55:29 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmp183tk0k9/envs/virtualenv_envs/mlflow-3371859241c0371f405bc46b1b817b0ba908bc2c/bin/activate && python /Users/mohammedanas/Learning/Machine Learning/.venv/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///var/folders/p2/jnw2fq7d2ybgc_

MlflowException: Non-zero exit code: 2
Command: ['bash', '-c', 'source /var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmp183tk0k9/envs/virtualenv_envs/mlflow-3371859241c0371f405bc46b1b817b0ba908bc2c/bin/activate && python /Users/mohammedanas/Learning/Machine Learning/.venv/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmpiz4zkp0t/iris_model --content-type json --input-path /var/folders/p2/jnw2fq7d2ybgc_d7k00nhlsh0000gn/T/tmpcuuukqbo/input.json']

An exception occurred while running model prediction within a uv environment. You can find the error message from the prediction subprocess by scrolling above.

## Load the model back for prediction as generic python function model. 

In [45]:
loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)
# Make predictions with the loaded model
predictions = loaded_model.predict(X_test)

iris_features = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features)
result["Actual Class"] = y_test
result["predictions"] = predictions



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Actual Class,predictions
0,6.7,3.3,5.7,2.5,2,2
1,6.4,2.9,4.3,1.3,1,1
2,5.6,2.8,4.9,2.0,2,2
3,6.2,3.4,5.4,2.3,2,2
4,5.0,2.3,3.3,1.0,1,1
5,6.7,3.1,5.6,2.4,2,2
6,7.2,3.0,5.8,1.6,2,2
7,6.5,3.2,5.1,2.0,2,2
8,6.9,3.1,5.1,2.3,2,2
9,5.4,3.9,1.7,0.4,0,0


In [44]:
import mlflow
logged_model = 'runs:/88e6163b03544a34954e46e9ea2a7224/iris_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

array([2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2,
       1, 0, 0, 2, 0, 2, 1, 2])

## Model Registry

The MLFlow model registry component is a centralized model store, set of API's and UI's to collaboratively manage the full life cycle of an MLFlow model. It provides model lineage (which MLFlow experiment and run produced the model), model versioning, model aliasing, model tagging and annotations. 

In [52]:
with mlflow.start_run():
    # Log model hyperparameters
    mlflow.log_params(params)

    # Log model accuracy metrics
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training info", "Basic LR model for iris dataset")
    
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
    )

2025/05/21 15:06:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run wistful-bug-735 at: http://127.0.0.1:5000/#/experiments/709085634117900399/runs/858786dc9f64448bb4f2bcfb0771ace8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/709085634117900399


## Inferencing from model registry

In [54]:
import mlflow.sklearn

model_name = "Iris_Classification_Model"
model_version = "latest"
model_uri = f"models:/{model_name}/{model_version}"

load_model = mlflow.sklearn.load_model(model_uri=model_uri)
load_model

LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [55]:
y_pred_new = load_model.predict(X_test)
y_pred_new

array([2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2,
       1, 0, 0, 2, 0, 2, 1, 2])